In [1]:
import numpy as np
import pywt
from scipy import signal

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Hjorth

In [2]:
def hjorth_mean(eeg_data):
    """
    Calculate the Hjorth parameters of EEG data and return the mean values across channels for each epoch.
    Args:
        eeg_data (ndarray): EEG data of shape (number_of_epochs, number_of_channels, number_of_datapoints_per_epoch).
    Returns:
        mean_activity (ndarray): Mean activity parameter of shape (number_of_epochs,).
        mean_mobility (ndarray): Mean mobility parameter of shape (number_of_epochs,).
        mean_complexity (ndarray): Mean complexity parameter of shape (number_of_epochs,).
    """
    n_epochs, n_channels, n_datapoints = eeg_data.shape
    mean_activity = np.zeros((n_epochs,))
    mean_mobility = np.zeros((n_epochs,))
    mean_complexity = np.zeros((n_epochs,))
    for i in range(n_epochs):
        activity = 0
        mobility = 0
        complexity = 0
        for j in range(n_channels):
            signal = eeg_data[i, j, :]
            diff1 = np.diff(signal)
            diff2 = np.diff(signal, n=2)
            var_zero = np.var(signal)
            var_d1 = np.var(diff1)
            var_d2 = np.var(diff2)
            activity += var_zero
            mobility += np.sqrt(var_d1 / var_zero)
            complexity += np.sqrt(var_d2 / var_d1) / np.sqrt(var_d1 / var_zero)
        mean_activity[i] = activity / n_channels
        mean_mobility[i] = mobility / n_channels
        mean_complexity[i] = complexity / n_channels
    return mean_activity, mean_mobility, mean_complexity

## Kurtosis

In [32]:
def kurtosis_feature(eeg_data):
    num_epochs, num_channels, num_datapoints_per_epoch = eeg_data.shape
    result = np.zeros(num_epochs)
    for i in range(num_epochs):
        epoch_data = eeg_data[i, :, :]
        epoch_mean = np.mean(epoch_data, axis=1)
        epoch_std = np.std(epoch_data, axis=1, ddof=1)
        epoch_kurtosis = (
            np.mean((epoch_data.T - epoch_mean) ** 4, axis=0) / epoch_std**4 - 3
        )
        result[i] = np.mean(epoch_kurtosis)
    return result

## Wavelet Fetures!
### Approx Mean, Approx Std Deviation, Approx Energy, Detailed Mean, Detailed Std Deviation, Detailed Energy, Approx Entropy & Detailed Entropy

In [4]:
def wavelet_features(epoch):
    num_epochs, num_channels, num_samples = epoch.shape
    cA_values = np.zeros((num_epochs, num_channels, num_samples // 2))
    cD_values = np.zeros((num_epochs, num_channels, num_samples // 2))
    cA_mean = np.zeros((num_epochs, num_channels))
    cA_std = np.zeros((num_epochs, num_channels))
    cA_Energy = np.zeros((num_epochs, num_channels))
    cD_mean = np.zeros((num_epochs, num_channels))
    cD_std = np.zeros((num_epochs, num_channels))
    cD_Energy = np.zeros((num_epochs, num_channels))
    Entropy_D = np.zeros((num_epochs, num_channels))
    Entropy_A = np.zeros((num_epochs, num_channels))
    wfeatures = np.zeros((num_epochs, 7 * num_channels))

    for i in range(num_epochs):
        for j in range(num_channels):
            cA, cD = pywt.dwt(epoch[i, j, :], "coif1")
            # cA_values[i, j, :] = cA
            # cD_values[i, j, :] = cD
            cA_mean[i, j] = np.mean(cA)
            cA_std[i, j] = np.abs(np.std(cA))
            cA_Energy[i, j] = np.sum(np.square(cA))
            cD_mean[i, j] = np.mean(cD)
            cD_std[i, j] = np.abs(np.std(cD))
            cD_Energy[i, j] = np.sum(np.square(cD))
            Entropy_D[i, j] = np.sum(np.square(cD) * np.log(np.square(cD)))
            Entropy_A[i, j] = np.sum(np.square(cA) * np.log(np.square(cA)))

    wfeatures[:, 0::7] = cA_mean
    wfeatures[:, 1::7] = cA_std
    wfeatures[:, 2::7] = cA_Energy
    wfeatures[:, 3::7] = cD_mean
    wfeatures[:, 4::7] = cD_std
    wfeatures[:, 5::7] = cD_Energy
    wfeatures[:, 6::7] = Entropy_D + Entropy_A

    return wfeatures

## Power Spectral Density

In [5]:
def maxPwelch_epochs(epochs, Fs):
  n_epochs, n_channels, n_samples_per_epoch = epochs.shape
  BandF = [12, 30, 100]
  PMax = np.zeros([n_epochs, n_channels, len(BandF) - 1])

  for i in range(n_epochs):
    for j in range(n_channels):
      f, Psd = signal.welch(epochs[i, j, :], Fs)

      if np.any(np.isnan(Psd)):
        nonnan_values = Psd[~np.isnan(Psd)]
        nan_average = np.mean(nonnan_values)
        Psd[np.isnan(Psd)] = nan_average

      for k in range(len(BandF) - 1):
        fr = np.where((f > BandF[k]) & (f <= BandF[k + 1]))
        PMax[i, j, k] = np.max(Psd[fr])

  return PMax

## Loading preporcessed Data from file

In [8]:
# Run this cell for executing preprocessing.v1
epoch_data = np.load("./cleaned_data.npy")

In [52]:
# Run this cell for executing preprocessing.v2
epoch_data = np.load("../codes-v2/filtered_data_v2.npy")

In [6]:
# Run this cell for executing preprocessing.v2 with all 29 subjects
epoch_data = np.load("../codes-v2/filtered_full_data_v1.npy")

In [15]:
# Run this cell for executing preprocessing.v2 with four class classification
epoch_data = np.load("../codes-v2/filtered_data_v3.npy")

In [54]:
# Run for feature extraction of test-subjects
subject_number = 'twofour'

epoch_data = np.load(f"../test-subjects/filtered-sub-{subject_number}.npy")

In [55]:
display(epoch_data.shape)

(2882, 14, 500)

In [56]:
hjorth = hjorth_mean(epoch_data)

In [57]:
hjorth_list = np.concatenate(
    (hjorth[0][:, np.newaxis], hjorth[1][:, np.newaxis], hjorth[2][:, np.newaxis]),
    axis=1,
)
display(hjorth_list.shape)

(2882, 3)

In [58]:
display(hjorth[0].shape, hjorth[1].shape, hjorth[2].shape)

(2882,)

(2882,)

(2882,)

In [59]:
kurtosis = kurtosis_feature(epoch_data)
display(kurtosis.shape)

(2882,)

In [60]:
wavelet = wavelet_features(epoch_data)
display(wavelet.shape)

(2882, 98)

In [61]:
def flatten(data):
    flattened_data = data.reshape(data.shape[0], -1)
    return flattened_data

In [62]:
psd = maxPwelch_epochs(epoch_data, 500)
display(psd.shape)

(2882, 14, 2)

In [63]:
psd_2d = flatten(psd)
display(psd_2d.shape)

(2882, 28)

In [17]:
# Run this cell to save features extracted for all 29 subjects
np.save("../all-sub-features/hjorth_list.npy", hjorth_list)
np.save("../all-sub-features/kurtosis.npy", kurtosis)
np.save("../all-sub-features/wavelet.npy", wavelet)
np.save("../all-sub-features/psd_2d.npy", psd_2d)

### Adding labels and building the final feature vector

In [64]:
# Run this cell for test-subjects
labels = np.load(f"../test-subjects/label-sub-{subject_number}.npy")
display(labels.shape)

(2882,)

In [ ]:
# Run this cell when executing preprocessing.v1
labels = np.load("./labels.npy")
display(labels.shape)

In [46]:
# Run this cell when executing preprocessing.v2
labels = np.load("../codes-v2/labels_v2.npy")
display(labels.shape)

(8391,)

In [5]:
# Run this cell when executing preprocessing.v2 with all 29 subjects
labels = np.load("../codes-v2/labels_full_v1.npy")
display(labels.shape)

(24354,)

In [19]:
# Run this cell when executing preprocessing.v2 with four class
labels = np.load("../codes-v2/labels_v3.npy")
display(labels.shape)

(16827,)

### Building Feature Vector for ML

In [3]:
# Run this cell to load features extracted for all 29 subjects
hjorth_list = np.load("../all-sub-features/hjorth_list.npy")
kurtosis = np.load("../all-sub-features/kurtosis.npy")
wavelet= np.load("../all-sub-features/wavelet.npy")
psd_2d = np.load("../all-sub-features/psd_2d.npy")

In [65]:
feature_vector = np.concatenate(
    (
      # hjorth_list, 
      # kurtosis[:, np.newaxis], 
      wavelet, 
      # psd_2d,
      labels[:, np.newaxis]
    ),
    axis=1,
)
display(feature_vector.shape)

(2882, 99)

In [66]:
# Run this cell when doing feature extration for test subjects
np.save(f"../test-subjects/wavelet-sub-{subject_number}.npy", feature_vector)

In [28]:
# Run this cell when doing feature extraction specific to features for all 29 subjects
np.save("../all-sub-features/rest-2-all.npy", feature_vector)

In [ ]:
# Run this cell when executing Preprocessing.v1
np.save("labeled_feature_vector.npy", feature_vector)

In [87]:
# Run this cell when doing feature extraction specific to features
np.save("../codes-v2/labeled_feature_rest-2-all.npy", feature_vector)

In [25]:
# Run this cell when executing preprocessing.v2 with four class
np.save("../codes-v2/labeled_feature_v3.npy", feature_vector)

In [64]:
display(feature_vector[:5, -1], feature_vector[-5:, -1])

array([0., 0., 0., 0., 0.])

array([1., 1., 1., 1., 1.])

In [35]:
def encoder(str):
  switcher = {
    "rest" : 0,
    "zerob" : 1,
    "oneb" : 2,
    "twob" : 3,
  }
  return switcher.get(str, "error")

In [36]:
def filter(feature_vector, label_1, label_2):
  l1 = encoder(label_1)
  l2 =encoder(label_2)
  feature_filter = feature_vector[np.logical_or(feature_vector[:, -1]==l1, feature_vector[:, -1]==l2)]
  return feature_filter

In [37]:
feature_filter = filter(feature_vector, 
                        "rest", 
                        "twob"
                        )
display(feature_filter.shape, feature_filter[:5, -1], feature_filter[-5:, -1])
print(feature_filter[4150:4200, -1])

(8391, 32)

array([0., 0., 0., 0., 0.])

array([3., 3., 3., 3., 3.])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3.]


In [38]:
# Run this cell when executing preprocessing.v2 with four class and filtering two classes
np.save("../codes-v2/labeled_feature_v2.npy", feature_filter)

### Building Feature Vector for Deep Learning

In [17]:
feature_vector = np.concatenate(
    (hjorth_list, kurtosis[:, np.newaxis], wavelet, psd_2d),
    axis=1,
)

In [21]:
print(feature_vector[:10])

[[5.91860190e-11 6.06506149e-01 1.95788105e+00 ... 4.22156961e-12
  5.93556943e-13 2.53393562e-12]
 [4.27787090e-11 6.56674113e-01 1.95179283e+00 ... 7.45609870e-13
  3.37528040e-13 3.89414158e-13]
 [4.07509746e-11 7.21396852e-01 1.76879551e+00 ... 8.25299746e-13
  7.10077633e-13 2.79019337e-13]
 ...
 [3.92580840e-11 6.68710581e-01 1.99244923e+00 ... 4.73538717e-13
  5.43931866e-13 2.58039600e-13]
 [4.09389814e-11 6.68757648e-01 2.02365791e+00 ... 4.73755447e-13
  9.34644892e-13 5.62357637e-13]
 [4.22822163e-11 6.42625041e-01 2.12024910e+00 ... 2.96167865e-13
  5.74304045e-13 3.07774333e-13]]


In [18]:
display(feature_vector.shape)

(16827, 130)

In [19]:
# Run this cell when applying DL with four class
np.save("../codes-v2/dl_feature_c4.npy", feature_vector)